# SMS Spam Detection
### by Jason Paul Miller
#### 03/14/23 PI Day

# Important Imports

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None  # removes false positive warning
import matplotlib.pyplot as plt
import re # Text cleanup
import string # Text cleanup
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'pandas'

### Read in the data

In [ ]:
data = pd.read_csv('spam.csv', encoding='latin-1')
data

### Clean up the data

In [ ]:
# Remove unnecessary columns
data = data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
print(data.shape)
data